In [1]:
import scrapy
from scrapy.crawler import CrawlerProcess
import warnings
warnings.filterwarnings('ignore')

In [2]:
import logging

class GERdailyZuwachs(scrapy.Spider):
    name = 'corona_gerdaily'
    start_urls = ['https://de.wikipedia.org/wiki/COVID-19-Pandemie_in_Deutschland/Statistik']

    def parse(self, response):
        for bundesland in response.css('table.wikitable > tbody > tr'):    #.zebra.toptextcells.mw-collapsible.mw-made-collapsible

                yield{
                    
                  'BW' : bundesland.xpath('td[2]//text()').extract_first(),
                  'BY' : bundesland.xpath('td[3]//text()').extract_first(),
                  'BE' : bundesland.xpath('td[4]//text()').extract_first(),
                  'BB' : bundesland.xpath('td[5]//text()').extract_first(),
                  'HB' : bundesland.xpath('td[6]//text()').extract_first(),
                  'HH' : bundesland.xpath('td[7]//text()').extract_first(),
                  'HE' : bundesland.xpath('td[8]//text()').extract_first(),
                  'MV' : bundesland.xpath('td[9]//text()').extract_first(),
                  'NI' : bundesland.xpath('td[10]//text()').extract_first(),
                  'NW' : bundesland.xpath('td[11]//text()').extract_first(),
                  'RP' : bundesland.xpath('td[12]//text()').extract_first(),
                  'SL' : bundesland.xpath('td[13]//text()').extract_first(),
                  'SN' : bundesland.xpath('td[14]//text()').extract_first(),
                  'ST' : bundesland.xpath('td[15]//text()').extract_first(),
                  'SH' : bundesland.xpath('td[16]//text()').extract_first(),
                  'TH' : bundesland.xpath('td[17]//text()').extract_first(),
                  'Gesamt' : bundesland.xpath('td[18]//text()').extract_first(),
               }

process = CrawlerProcess({
'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)',
'FEED_FORMAT': 'csv',
'FEED_URI': 'de_zuwachsneu.csv'
})

process.crawl(GERdailyZuwachs)
process.start()

2020-10-29 13:57:46 [scrapy.utils.log] INFO: Scrapy 2.3.0 started (bot: scrapybot)
2020-10-29 13:57:46 [scrapy.utils.log] INFO: Versions: lxml 4.5.2.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.5.2, w3lib 1.21.0, Twisted 20.3.0, Python 3.7.7 (default, May  6 2020, 11:45:54) [MSC v.1916 64 bit (AMD64)], pyOpenSSL 19.1.0 (OpenSSL 1.1.1h  22 Sep 2020), cryptography 3.1.1, Platform Windows-10-10.0.19041-SP0
2020-10-29 13:57:46 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2020-10-29 13:57:46 [scrapy.crawler] INFO: Overridden settings:
{'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'}
2020-10-29 13:57:46 [scrapy.extensions.telnet] INFO: Telnet Password: 46d1abfab60aa4a2
2020-10-29 13:57:46 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.logstats.LogStats']
2020-10-29 13:57:46 [scrapy.mi

2020-10-29 13:57:46 [scrapy.core.scraper] DEBUG: Scraped from <200 https://de.wikipedia.org/wiki/COVID-19-Pandemie_in_Deutschland/Statistik>
{'BW': '9.\xa0März 2020\n', 'BY': '204\n', 'BE': '256\n', 'BB': '48\n', 'HB': '6\n', 'HH': '4\n', 'HE': '17\n', 'MV': '26\n', 'NI': '10\n', 'NW': '38\n', 'RP': '484\n', 'SL': '17\n', 'SN': '6\n', 'ST': '12\n', 'SH': '—\n', 'TH': '9\n', 'Gesamt': '2\n'}
2020-10-29 13:57:46 [scrapy.core.scraper] DEBUG: Scraped from <200 https://de.wikipedia.org/wiki/COVID-19-Pandemie_in_Deutschland/Statistik>
{'BW': '237\n', 'BY': '314\n', 'BE': '48\n', 'BB': '9\n', 'HB': '4\n', 'HH': '29\n', 'HE': '35\n', 'MV': '13\n', 'NI': '49\n', 'NW': '484\n', 'RP': '25\n', 'SL': '7\n', 'SN': '22\n', 'ST': '7\n', 'SH': '9\n', 'TH': '4\n', 'Gesamt': '1.296\n'}
2020-10-29 13:57:46 [scrapy.core.scraper] DEBUG: Scraped from <200 https://de.wikipedia.org/wiki/COVID-19-Pandemie_in_Deutschland/Statistik>
{'BW': '277\n', 'BY': '366\n', 'BE': '90\n', 'BB': '24\n', 'HB': '21\n', 'HH': '4

2020-10-29 13:57:46 [scrapy.core.scraper] DEBUG: Scraped from <200 https://de.wikipedia.org/wiki/COVID-19-Pandemie_in_Deutschland/Statistik>
{'BW': '30.\xa0März 2020\n', 'BY': '10.943\n', 'BE': '13.989\n', 'BB': '2.464\n', 'HB': '761\n', 'HH': '286\n', 'HE': '2.053\n', 'MV': '3.091\n', 'NI': '356\n', 'NW': '3.732\n', 'RP': '12.178\n', 'SL': '2.584\n', 'SN': '706\n', 'ST': '1.795\n', 'SH': '592', 'TH': '1.049\n', 'Gesamt': '719\n'}
2020-10-29 13:57:46 [scrapy.core.scraper] DEBUG: Scraped from <200 https://de.wikipedia.org/wiki/COVID-19-Pandemie_in_Deutschland/Statistik>
{'BW': '12.334\n', 'BY': '14.810\n', 'BE': '2.575\n', 'BB': '798\n', 'HB': '294\n', 'HH': '2.191\n', 'HE': '3.283\n', 'MV': '366\n', 'NI': '4.063\n', 'NW': '13.225\n', 'RP': '2.726\n', 'SL': '782\n', 'SN': '1.882\n', 'ST': '680\n', 'SH': '1.120\n', 'TH': '784\n', 'Gesamt': '61.913\n'}
2020-10-29 13:57:46 [scrapy.core.scraper] DEBUG: Scraped from <200 https://de.wikipedia.org/wiki/COVID-19-Pandemie_in_Deutschland/Statisti

2020-10-29 13:57:46 [scrapy.core.scraper] DEBUG: Scraped from <200 https://de.wikipedia.org/wiki/COVID-19-Pandemie_in_Deutschland/Statistik>
{'BW': '27.258\n', 'BY': '36.881\n', 'BE': '5.066\n', 'BB': '2.161\n', 'HB': '567\n', 'HH': '4.118\n', 'HE': '6.916\n', 'MV': '645\n', 'NI': '8.649\n', 'NW': '28.006\n', 'RP': '5.324\n', 'SL': '2.289\n', 'SN': '4.140\n', 'ST': '1.315\n', 'SH': '2.387\n', 'TH': '1.717\n', 'Gesamt': '137.439\n'}
2020-10-29 13:57:46 [scrapy.core.scraper] DEBUG: Scraped from <200 https://de.wikipedia.org/wiki/COVID-19-Pandemie_in_Deutschland/Statistik>
{'BW': '27.883\n', 'BY': '37.407\n', 'BE': '5.159\n', 'BB': '2.221\n', 'HB': '585\n', 'HH': '4.167\n', 'HE': '7.108\n', 'MV': '651\n', 'NI': '8.797\n', 'NW': '28.471\n', 'RP': '5.432\n', 'SL': '2.303\n', 'SN': '4.190\n', 'ST': '1.350\n', 'SH': '2.417\n', 'TH': '1.756\n', 'Gesamt': '139.897\n'}
2020-10-29 13:57:46 [scrapy.core.scraper] DEBUG: Scraped from <200 https://de.wikipedia.org/wiki/COVID-19-Pandemie_in_Deutschlan

2020-10-29 13:57:47 [scrapy.core.scraper] DEBUG: Scraped from <200 https://de.wikipedia.org/wiki/COVID-19-Pandemie_in_Deutschland/Statistik>
{'BW': '32.762\n', 'BY': '43.658\n', 'BE': '6.149\n', 'BB': '2.999\n', 'HB': '963\n', 'HH': '4.704\n', 'HE': '8.736\n', 'MV': '715\n', 'NI': '10.564\n', 'NW': '34.249\n', 'RP': '6.213\n', 'SL': '2.655\n', 'SN': '4.836\n', 'ST': '1.602\n', 'SH': '2.834\n', 'TH': '2.452\n', 'Gesamt': '166.091\n'}
2020-10-29 13:57:47 [scrapy.core.scraper] DEBUG: Scraped from <200 https://de.wikipedia.org/wiki/COVID-19-Pandemie_in_Deutschland/Statistik>
{'BW': '32.898\n', 'BY': '43.905\n', 'BE': '6.209\n', 'BB': '3.037\n', 'HB': '997\n', 'HH': '4.753\n', 'HE': '8.850\n', 'MV': '718\n', 'NI': '10.686\n', 'NW': '34.522\n', 'RP': '6.241\n', 'SL': '2.658\n', 'SN': '4.839\n', 'ST': '1.615\n', 'SH': '2.860\n', 'TH': '2.512\n', 'Gesamt': '167.300\n'}
2020-10-29 13:57:47 [scrapy.core.scraper] DEBUG: Scraped from <200 https://de.wikipedia.org/wiki/COVID-19-Pandemie_in_Deutschl

...

2020-10-29 13:57:48 [scrapy.core.scraper] DEBUG: Scraped from <200 https://de.wikipedia.org/wiki/COVID-19-Pandemie_in_Deutschland/Statistik>
{'BW': '1,10\n', 'BY': '0,88–1,30\n', 'BE': '0,97\n', 'BB': '0,86–1,07\n', 'HB': None, 'HH': None, 'HE': None, 'MV': None, 'NI': None, 'NW': None, 'RP': None, 'SL': None, 'SN': None, 'ST': None, 'SH': None, 'TH': None, 'Gesamt': None}
2020-10-29 13:57:48 [scrapy.core.scraper] DEBUG: Scraped from <200 https://de.wikipedia.org/wiki/COVID-19-Pandemie_in_Deutschland/Statistik>
{'BW': '1,12\n', 'BY': '0,87–1,37\n', 'BE': '0,98\n', 'BB': '0,84–1,06\n', 'HB': None, 'HH': None, 'HE': None, 'MV': None, 'NI': None, 'NW': None, 'RP': None, 'SL': None, 'SN': None, 'ST': None, 'SH': None, 'TH': None, 'Gesamt': None}
2020-10-29 13:57:48 [scrapy.core.scraper] DEBUG: Scraped from <200 https://de.wikipedia.org/wiki/COVID-19-Pandemie_in_Deutschland/Statistik>
{'BW': '1,10\n', 'BY': '0,90–1,31\n', 'BE': '0,98\n', 'BB': '0,87–1,09\n', 'HB': None, 'HH': None, 'HE': No

2020-10-29 13:57:48 [scrapy.core.scraper] DEBUG: Scraped from <200 https://de.wikipedia.org/wiki/COVID-19-Pandemie_in_Deutschland/Statistik>
{'BW': '1,18\n', 'BY': '0,95–1,40\n', 'BE': '0,98\n', 'BB': '0,86–1,09\n', 'HB': None, 'HH': None, 'HE': None, 'MV': None, 'NI': None, 'NW': None, 'RP': None, 'SL': None, 'SN': None, 'ST': None, 'SH': None, 'TH': None, 'Gesamt': None}
2020-10-29 13:57:48 [scrapy.core.scraper] DEBUG: Scraped from <200 https://de.wikipedia.org/wiki/COVID-19-Pandemie_in_Deutschland/Statistik>
{'BW': '1,12\n', 'BY': '0,91–1,34\n', 'BE': '1,03\n', 'BB': '0,92–1,15\n', 'HB': None, 'HH': None, 'HE': None, 'MV': None, 'NI': None, 'NW': None, 'RP': None, 'SL': None, 'SN': None, 'ST': None, 'SH': None, 'TH': None, 'Gesamt': None}
2020-10-29 13:57:48 [scrapy.core.scraper] DEBUG: Scraped from <200 https://de.wikipedia.org/wiki/COVID-19-Pandemie_in_Deutschland/Statistik>
{'BW': '0,96\n', 'BY': '0,78–1,17\n', 'BE': '1,00\n', 'BB': '0,91–1,12\n', 'HB': None, 'HH': None, 'HE': No

2020-10-29 13:57:48 [scrapy.core.scraper] DEBUG: Scraped from <200 https://de.wikipedia.org/wiki/COVID-19-Pandemie_in_Deutschland/Statistik>
{'BW': '1,25\n', 'BY': '1,04–1,50\n', 'BE': '1,23\n', 'BB': '1,09–1,36\n', 'HB': None, 'HH': None, 'HE': None, 'MV': None, 'NI': None, 'NW': None, 'RP': None, 'SL': None, 'SN': None, 'ST': None, 'SH': None, 'TH': None, 'Gesamt': None}
2020-10-29 13:57:48 [scrapy.core.scraper] DEBUG: Scraped from <200 https://de.wikipedia.org/wiki/COVID-19-Pandemie_in_Deutschland/Statistik>
{'BW': '1,09\n', 'BY': '0,92–1,29\n', 'BE': '1,17\n', 'BB': '1,06–1,28\n', 'HB': None, 'HH': None, 'HE': None, 'MV': None, 'NI': None, 'NW': None, 'RP': None, 'SL': None, 'SN': None, 'ST': None, 'SH': None, 'TH': None, 'Gesamt': None}
2020-10-29 13:57:48 [scrapy.core.scraper] DEBUG: Scraped from <200 https://de.wikipedia.org/wiki/COVID-19-Pandemie_in_Deutschland/Statistik>
{'BW': '1,11\n', 'BY': '0,93–1,32\n', 'BE': '1,23\n', 'BB': '1,11–1,36\n', 'HB': None, 'HH': None, 'HE': No

2020-10-29 13:57:48 [scrapy.core.scraper] DEBUG: Scraped from <200 https://de.wikipedia.org/wiki/COVID-19-Pandemie_in_Deutschland/Statistik>
{'BW': '25.–31. Mai', 'BY': '0', 'BE': '00', 'BB': '178 ', 'HB': ' 1.017.179', 'HH': '143\n', 'HE': None, 'MV': None, 'NI': None, 'NW': None, 'RP': None, 'SL': None, 'SN': None, 'ST': None, 'SH': None, 'TH': None, 'Gesamt': None}
2020-10-29 13:57:48 [scrapy.core.scraper] DEBUG: Scraped from <200 https://de.wikipedia.org/wiki/COVID-19-Pandemie_in_Deutschland/Statistik>
{'BW': '1.–7. Juni', 'BY': '0', 'BE': '00', 'BB': '176 ', 'HB': ' 1.083.345', 'HH': '137\n', 'HE': None, 'MV': None, 'NI': None, 'NW': None, 'RP': None, 'SL': None, 'SN': None, 'ST': None, 'SH': None, 'TH': None, 'Gesamt': None}
2020-10-29 13:57:48 [scrapy.core.scraper] DEBUG: Scraped from <200 https://de.wikipedia.org/wiki/COVID-19-Pandemie_in_Deutschland/Statistik>
{'BW': '8.–14. Juni', 'BY': '0', 'BE': '00', 'BB': '172 ', 'HB': ' 1.092.448', 'HH': '139\n', 'HE': None, 'MV': None, 

2020-10-29 13:57:48 [scrapy.core.scraper] DEBUG: Scraped from <200 https://de.wikipedia.org/wiki/COVID-19-Pandemie_in_Deutschland/Statistik>
{'BW': '20.380.376\n', 'BY': '418.871 (2,06\xa0%)', 'BE': '\n', 'BB': None, 'HB': None, 'HH': None, 'HE': None, 'MV': None, 'NI': None, 'NW': None, 'RP': None, 'SL': None, 'SN': None, 'ST': None, 'SH': None, 'TH': None, 'Gesamt': None}
2020-10-29 13:57:48 [scrapy.core.engine] INFO: Closing spider (finished)
2020-10-29 13:57:48 [scrapy.extensions.feedexport] INFO: Stored csv feed (932 items) in: de_zuwachsneu.csv
2020-10-29 13:57:48 [scrapy.statscollectors] INFO: Dumping Scrapy stats:
{'downloader/request_bytes': 279,
 'downloader/request_count': 1,
 'downloader/request_method_count/GET': 1,
 'downloader/response_bytes': 90734,
 'downloader/response_count': 1,
 'downloader/response_status_count/200': 1,
 'elapsed_time_seconds': 2.264448,
 'finish_reason': 'finished',
 'finish_time': datetime.datetime(2020, 10, 29, 12, 57, 48, 807343),
 'item_scrape

In [3]:
import pandas as pd
import numpy as np
de_zuwachs = pd.read_csv('de_zuwachs4.csv', skiprows = range(1,547), nrows = 105 , thousands = '.' ) #, dtype={'BW': int } )
de_zuwachs['Datum'] = pd.date_range(start='24/2/2020', periods=len(de_zuwachs), freq='D')

#löst das Problem mit dem falschen Datum
first_col = de_zuwachs.pop('Datum')
de_zuwachs.insert(0, 'Datum', first_col)

#Ändert die NAN und - zu 0
de_zuwachs = de_zuwachs.fillna(0)
de_zuwachs = de_zuwachs.replace('—',0)

In [4]:
de_tode = pd.read_csv('de_zuwachs.csv', skiprows = range(1,735), nrows = 93, thousands = '.') 

de_tode['Datum'] = pd.date_range(start='3/9/2020', periods=len(de_tode), freq='D')
first_col = de_tode.pop('Datum')
de_tode.insert(0, 'Datum', first_col)

#Ändert die NAN und - zu 0
de_tode = de_tode.fillna(0)
de_tode = de_tode.replace('–',0)

In [5]:
welt_zuwachs = pd.read_csv('WHO-COVID-19-global-data.csv')
welt_zuwachs.drop(['Country_code','WHO_region'], axis=1, inplace=True)
welt_zuwachs['Date_reported'] = welt_zuwachs['Date_reported'].replace('T00:00:00Z'," ",regex = True)
#welt_zuwachs.head()
arraywelt = ["Germany"]
de_zuwachs = welt_zuwachs.loc[welt_zuwachs['Country'].isin(arraywelt)]

In [6]:
kreise_zuwachs = pd.read_csv('COVID-19 Cases Germany_1591375436907.csv')
kreise_zuwachs.drop(['relative_case_changes','fb_id','cases_per_population','relative_death_changes','bundesland_ags','kreis_ags','kreis_nuts','kreis_name','deaths_per_population','fb_datetime'], axis=1, inplace=True)
kreise_zuwachs = kreise_zuwachs.fillna(0)
kreise_zuwachs['deaths'] = kreise_zuwachs['deaths'].astype(int)
kreise_zuwachs['publication_datetime'] = kreise_zuwachs['publication_datetime'].str[:11]

In [7]:
laendereinwohner = pd.read_csv('population-figures-by-country-csv_csv.csv')
laendereinwohner = laendereinwohner[['Country','Year_2016']]
laendereinwohner = laendereinwohner.fillna(0)
laendereinwohner['Year_2016'] = laendereinwohner['Year_2016'].astype(int)

In [8]:
bundeslandeinwohner = pd.read_csv('bundeslandeinw.csv',skiprows= range(1,2), nrows = 16, thousands = '.')

In [9]:
kreise_zuwachs = kreise_zuwachs.sort_values(['location_label','publication_datetime'])
kreise_zuwachs['täglicher Wachstum'] = kreise_zuwachs['cases'].pct_change()

In [10]:
counties = pd.read_csv('covid_confirmed_usafacts.csv')
countyAC = counties.loc[(counties['County Name'] == 'Albany County') & (counties['State'] == 'NY')]
countyAC.drop(['countyFIPS', 'County Name', 'State', 'stateFIPS'], axis=1, inplace=True)
countyAC = countyAC.T
countyAC['date'] = countyAC.index
countyAC['County_name'] = 'Albany County'
countyAC['State'] = 'NY'
countyAC['cases'] = countyAC[1863]
countyAC['date'] = pd.to_datetime(countyAC['date'], format='%m/%d/%y')
countyAC['dailycases'] = countyAC['cases'].diff()
countyAC['dailycases'][countyAC['dailycases'] < 0] = 0
countyAC.loc['1/22/20','dailycases'] = countyAC.loc['1/22/20', 'cases']
countyAC['kumcases'] = countyAC['dailycases'].cumsum()

In [11]:
# Erstellung von 10 DFs 5 kleine Landkreise und 5 große Landkreise

arraykreise = kreise_zuwachs['location_label'].tolist()
arraykreise = list(dict.fromkeys(arraykreise))  #Entfernt alle Dubletten
#arraykreise = ['Mettmann','Main-Taunus-Kreis','Offenbach','Esslingen','Recklinghausen','Altmarkkreis Salzwedel','Prignitz','Ostprignitz-Ruppin','Uckermark','Stendal']
arraykreise = sorted(arraykreise)
dflist = arraykreise
d = {} #dict damit können alle erstelleten DataFrames aufgerufen werden

for j in dflist:
   d[j] = pd.DataFrame()
i = 0

durchschnittlicheswachstum = []

for name, df in d.items():
        d[name] = kreise_zuwachs[kreise_zuwachs['location_label'] == arraykreise[i]]
        d[name] = d[name].drop(columns=['population','bundesland_name'])
        d[name]['dailycases'] = d[name]['cases'].diff()
        d[name]['publication_datetime'] = pd.to_datetime(d[name]['publication_datetime'])
        d[name] = d[name].sort_values(['publication_datetime'])
        d[name]['dailycases'][d[name]['dailycases'] < 0] = 0
        d[name]['Rate'] = d[name]['dailycases'].pct_change()
        d[name]['Rate'] = d[name]['Rate'].replace([np.inf, -np.inf], 0)
        d[name] = d[name].fillna(0)
        
        #10 DFs sind vorhanden nun muss durchschnittliche Wachstumsrate berechnet werden
        durchschnittlicheswachstum.append(d[name]['Rate'].sum()/len(d[name].index))
        i = i + 1

Hier wird mithilfe eines Dictionary für jeden Landkreis ein DF erstellt und dann eine Liste in der alle Durchschnittlichen Wachstumsraten aller Landkreise drin stehen

In [12]:
d['Hochsauerlandkreis'].loc[6941,'dailycases'] = d['Hochsauerlandkreis'].loc[6941, 'cases']
d['Hochsauerlandkreis']['kumcases'] = d['Hochsauerlandkreis']['dailycases'].cumsum()

d['Hochsauerlandkreis']['dailydeaths'] = d['Hochsauerlandkreis']['deaths'].diff()
d['Hochsauerlandkreis']['dailydeaths'][d['Hochsauerlandkreis']['dailydeaths'] < 0] = 0
d['Hochsauerlandkreis'].loc[6941,'dailydeaths'] = d['Hochsauerlandkreis'].loc[6941, 'deaths']
d['Hochsauerlandkreis']['kumdeaths'] = d['Hochsauerlandkreis']['dailydeaths'].cumsum()


In [13]:
d['Essen'].loc[6936,'dailycases'] = d['Essen'].loc[6936, 'cases']
d['Essen']['kumcases'] = d['Essen']['dailycases'].cumsum()

d['Essen']['dailydeaths'] = d['Essen']['deaths'].diff()
d['Essen']['dailydeaths'][d['Essen']['dailydeaths'] < 0] = 0
d['Essen'].loc[6936,'dailydeaths'] = d['Essen'].loc[6936, 'deaths']
d['Essen']['kumdeaths'] = d['Essen']['dailydeaths'].cumsum()


In [14]:
d['Hochsauerlandkreis']['Wochentag'] = d['Hochsauerlandkreis']['publication_datetime'].dt.day_name()
countyAC['Weekday'] = countyAC['date'].dt.day_name()
d['Essen']['Wochentag'] = d['Essen']['publication_datetime'].dt.day_name()

In [15]:
import locale
locale.setlocale(locale.LC_ALL, '')

'German_Germany.1252'

In [16]:
#Noch in bearbeitung
#bund_zuwachsraw = pd.read_csv('COVID-19 Cases Germany_1591375436907.csv')
#bund_zuwachsraw.drop(['deaths_per_100k','deaths','relative_case_changes','fb_id','cases_per_population','relative_death_changes','bundesland_ags','kreis_ags','kreis_nuts','kreis_name','deaths_per_population','fb_datetime'], axis=1, inplace=True)
#bund_zuwachsraw = bund_zuwachsraw.fillna(0)
#bund_zuwachsraw['publication_datetime'] = bund_zuwachsraw['publication_datetime'].str[:11]
#bund_zuwachsraw = bund_zuwachsraw.sort_values(['publication_datetime'])

In [17]:
#Noch in bearbeitung
#bundzuwachsraw = pd.read_csv('RKI_COVID19.csv')
#listSH = ["Schleswig-Holstein"]
#SH = bundzuwachsraw.loc[bundzuwachsraw['Bundesland'].isin(listSH)]
#SH['Meldedatum'] = pd.to_datetime(SH['Meldedatum'])

In [18]:
#Noch in bearbeitung
#import datetime

#date = datetime.datetime(2020, 6, 30)
#SHsum = 0

#for i in SH.index:
 #   if SH['Meldedatum'][i] == date:
 #       SHsum = SHsum + SH['AnzahlFall'][i]
 #   #date = date - timedelta(days=1)
        
#print(date)

In [19]:
from string import ascii_lowercase

bundzuwachsraw = pd.read_csv('table-1.csv')
bundzuwachsraw.set_index('Datum')
bundzuwachsraw = bundzuwachsraw.drop(['KW','Differenz zum Vortag','Bem.','Datum'],1) 
bundzuwachsraw = bundzuwachsraw.iloc[:-1]
bundzuwachsraw = bundzuwachsraw.fillna(0)
bundzuwachsraw = bundzuwachsraw.replace('—',0)

for column in bundzuwachsraw:
    bundzuwachsraw[column] = bundzuwachsraw[column].astype(str)
    bundzuwachsraw[column] = bundzuwachsraw[column].str.replace('.','')
    for c in ascii_lowercase:
        bundzuwachsraw[column] = bundzuwachsraw[column].str.replace(c,'')
    bundzuwachsraw[column] = bundzuwachsraw[column].str.replace(')','')
    bundzuwachsraw[column] = bundzuwachsraw[column].str.replace('(','')
    bundzuwachsraw[column] = bundzuwachsraw[column].astype(int)    

In [20]:
bundzuwachs = bundzuwachsraw.diff()
bundzuwachs.iloc[0] = bundzuwachsraw.iloc[0]
bundzuwachsraw['Date'] = pd.date_range(start='24/2/2020', periods=len(bundzuwachsraw), freq='D')        
bundzuwachs['Date'] = pd.date_range(start='24/2/2020', periods=len(bundzuwachsraw), freq='D')
bundzuwachsraw = bundzuwachsraw.set_index('Date')
bundzuwachs = bundzuwachs.set_index('Date')

In [21]:
bundchange = bundzuwachs.pct_change()
bundchange = bundchange.fillna(0)
bundchange = bundchange.replace([np.inf, -np.inf], 0) # Was sollte am besten hier hin für die Wachstumsrate?
#test3 # tägliche prozentuale Änderung der Coronazahlen

In [22]:
bundzuwachsraw_1 = bundzuwachsraw.iloc[:128] # Februar Tage + März + April + Mai + Juni
bundzuwachsraw_2 = bundzuwachsraw.iloc[128:]
bundzuwachs_1 = bundzuwachs.iloc[:128]
bundzuwachs_2 = bundzuwachs.iloc[128:]
bundchange_1 = bundchange.iloc[:128]
bundchange_2 = bundchange.iloc[128:]

In [23]:
durchschnittlichwachstum = bundchange_1.sum()/len(bundchange_1.index)
durchschnittlichwachstum = bundchange_1.sum()/len(bundchange_1.index)
durchwachstumfnl = pd.DataFrame({'Bundesland':durchschnittlichwachstum.index,'AGR':durchschnittlichwachstum.values})
#fnl = Final  AGR = average growth rate
liste = ['Baden-Württemberg','Bayern','Berlin','Brandenburg','Bremen','Hamburg','Hessen','Mecklenburg-Vorpommern','Niedersachsen','Nordrhein-Westfalen','Rheinland-Pfalz','Saarland','Sachsen','Sachsen-Anhalt','Schleswig-Holstein','Thüringen','Gesamt']

for i, row in durchwachstumfnl.iterrows():
        durchwachstumfnl.at[i,'Bundesland'] = liste[i]

In [24]:
dichte = pd.read_excel('A_BevDichte_Kreise.xlsx')
dichte = dichte.drop(columns = ['Kennziffer','Thünen-Typ'])
dichte = dichte.sort_values(by=['Name'])
dichte = dichte.reset_index()
dichte = dichte.drop(columns = 'index')

#Wird genutzt um die Stadt von dem Landkreis zu trennen
name = ' Stadt'

for j in range(len(dichte)):
    if j == 401:
        break
    else:
        if dichte['Name'][j] == dichte['Name'][j+1]:
            if dichte['Bevölkerungsdichte 2014'][j] > dichte['Bevölkerungsdichte 2014'][j+1]:
                dichte['Name'][j+1] = dichte['Name'][j+1] + name
            else:
                dichte['Name'][j] = dichte['Name'][j] + name

In [25]:
dichteU = pd.DataFrame(columns= ['Landkreis/Kreis','Bev.D.Ew./km²'])
dichteU['Landkreis/Kreis'] = arraykreise
dichteU = dichteU.fillna(0)

for i in range(len(dichte)):
    for j in range(len(arraykreise)):
        if arraykreise[j] == dichte['Name'][i]:
            dichteU['Bev.D.Ew./km²'][j] = dichte['Bevölkerungsdichte 2014'][i]

In [26]:
t = {'Landkreise/Kreis':arraykreise,'AGR':durchschnittlicheswachstum}
compLK = pd.DataFrame(t) #compLK = Vergleich der Landkreise von Wachstum und Bevölkerungsdichte
compLK['Bev.D.Ew./km²'] = dichteU['Bev.D.Ew./km²'].astype(int)
compLK = compLK[compLK['Bev.D.Ew./km²'] != 0]
compLKfnl = compLK.drop(columns =['Landkreise/Kreis'])